In [ ]:
import pandas as pd
import numpy as np
import folium
import matplotlib as plt
import sys
from pandas import DataFrame
from pandas import read_csv, read_excel
import seaborn as sns

train = pd.read_excel('안전비상벨위치정보.xlsx')

# 그래프 한글 설정
plt.rcParams["font.family"] = 'AppleGothic' if sys.platform == 'darwin' else 'Malgun Gothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.unicode_minus"] = False

df = train[['WGS84위도', 'WGS84경도']]
df = df.iloc[0:1500, :]
df = df.to_numpy()


from haversine import haversine ## 최단거리 구하는 모듈

# 위경도 입력
one = (df[0][0], df[0][1])  #Latitude, Longitude
two = (df[1][0], df[1][1])
three = (df[2][0], df[3][1])
four = (df[3][0], df[3][1])
# 거리 계산
haversine(four, three, unit = 'km')

import numpy as np
from scipy.spatial import distance_matrix ## 유클리디언 거리행렬 구해주는 
from gurobupy -> 최적화문제해결해주는 모듈임
from scipy.spatilal import ConvexHull
## convexhull : 평면위에 존재하는 집합 x 전체를 감싸기 위해 필요한 최소한의 부분집합
## 각 클러스터에서 가장 바깥에 있는 점을 의미함.



def generate_candidate_sites(points,M=100): # 후보지선정.
    '''
    Generate M candidate sites with the convex hull of a point set
    Input:
        points: a Numpy array with shape of (N,2) : 좌표값 행렬
        M: the number of candidate sites to generate : 후보지를 선정할 갯수
    Return:
        sites: a Numpy array with shape of (M,2)
    '''
	convexhull의 영역내부에서 m개의 후보지를 선정합니다

    hull = ConvexHull(points) # 위경도 값들로 하여금 하나의 클러스터를 생성
    plolygon_points = points[hull.vertices] # 그래프 즉 클러스터의 꼭지접에 대한 좌표를 얻어내는 것같음.

    poly = Polygon(polygon_points) # 꼭짓점에 대해 다격형 샹성
    min_x, min_y, max_x, max_y = poly.bounds # 경계인 좌표임

    sites = [] ## 후보지 리스트
    
    while len(sites)< M : 
        random_point = Point([random.uniform(min_x,max_x),
                          random.uniform(min_y,max_y)]) # 경계가 되는 점 안에서 랜덤한 좌표 생성

        if (random_point.within(poly)) : 
            sites.append(random_point) ## 랜덤한 좌표가 다각형 안에 있으면 후보지 리스트에 추가함.
    return np.array([p.x,p.y] for p in sites)


def MCLP(points,K.,radius,M) : 
    """
    Solve maximum covering location problem
    Input:
        points: input points, Numpy array in shape of [N,2] ## 좌표값 행렬
        K: the number of sites to select ## 최종 입지
        radius: the radius of circle ## 반지름
        M: the number of candidate sites, which will randomly generated inside # 후보지 수
        the ConvexHull wrapped by the polygon
    Return:
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        f: the optimal value of the objective function
    """
    print('----- Configurations -----')
    print('  Number of points %g' % points.shape[0]) 
    print('  K %g' % K)
    print('  Radius %g' % radius)
    print('  M %g' % M)
    import time
    start = time.time()
    sites = generate_candidate_sites(points,M) ## 후보지 선정
    J = sites.shape[0] # 후보지 갯수
    I = points.shape[0] # input 좌표 수 
    D = distance_matrix(points,sites) # 후보지와 input들의 거리행렬ㄹ
    mask1 = D<=radius # 커버범위 안에 있냐 
    D[mask1]=1
    D[~mask1]=0
    # Build model
    m = Model()
    # Add variables
    x = {}
    y = {}
    for i in range(I):
        y[i] = m.addVar(vtype=GRM.BINARY,name='y%d'% i ) # input데이터에 대해 종속변수 생성 (이진형임)
    for j in range(J): 
        x[j] = m.addVar(vtype=GRM.BINARY,name = 'y%d' % j) # 선정된 후보군애 대한 이진형 독립변수 생성

    m.update()
    # Add constraints
    m.addConstr(quicksum(x[j] for j in range(J)) == K) ## 제한조건 추가 : 설정한 k와 후보지 수와 같게끔 설정

    for i in range(I):
        m.addConstr(quicksum(x[j] for j in np.where(D[i]==1)[0])>= y[i]) # 먼말이야

    
    m.setObjective(quicksum(y[i]for i in range(I)),GRB.MAXIMIZE) # 커버범위를 최대화 하도록 목적함수를 정의.
    m.setParam('OutputFlag', 0)
    m.optimize()
    print('----- Output -----')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objVal)

    solution = []
    if m.status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            if v.x==1 and v.varName[0]=='x':
               solution.append(int(v.varName[1:]))
    opt_sites = sites[solution]
    return opt_sites,m.objVal

def plot_input(points):
    '''
    Plot the result
    Input:
        points: input points, Numpy array in shape of [N,2]
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        radius: the radius of circle
    '''
    from matplotlib import pyplot as plt
    fig = plt.figure(figsize=(8,8))
    plt.scatter(points[:,0],points[:,1],c='C0')
    ax = plt.gca()
    ax.axis('equal')
    ax.tick_params(axis='both',left=False, top=False, right=False,
                       bottom=False, labelleft=False, labeltop=False,
                       labelright=False, labelbottom=False)

def plot_result(points,opt_sites,radius):
    '''
    Plot the result
    Input:
        points: input points, Numpy array in shape of [N,2]
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        radius: the radius of circle
    '''
    from matplotlib import pyplot as plt
    fig = plt.figure(figsize=(8,8))
    plt.scatter(points[:,0],points[:,1],c='C0')
    ax = plt.gca()
    plt.scatter(opt_sites[:,0],opt_sites[:,1],c='C1',marker='+')
    for site in opt_sites:
        circle = plt.Circle(site, radius, color='C1',fill=False,lw=2)
        ax.add_artist(circle)
    ax.axis('equal')
    ax.tick_params(axis='both',left=False, top=False, right=False,
                       bottom=False, labelleft=False, labeltop=False,
                       labelright=False, labelbottom=False)